In [ ]:
%run <Fundraising_D365_Config>

# Initialize Synchronizers

Create sync instances once for reuse across multiple operations.

In [ ]:
# Initialize optionset synchronizer (used for all optionset syncs)
optionset_sync = Dynamics365OptionsetSync(
    source_id=source_id,
    source_lakehouse=bronze_lakehouse_name,
    target_lakehouse=silver_lakehouse_name
)

# Initialize data synchronizer (used for all data table syncs)
data_sync = Dynamics365DataSync(
    source_id=source_id,
    source_lakehouse=bronze_lakehouse_name,
    target_lakehouse=silver_lakehouse_name
)

## Constants

Business constants and configuration values used throughout the transformations.
Centralized to avoid magic numbers and improve maintainability.

In [ ]:
# ============================================================================
# BUSINESS CONSTANTS
# ============================================================================

# --- ConstituentType ---
CONSTITUENT_TYPE_INDIVIDUAL = "Individual"
CONSTITUENT_TYPE_ORGANIZATION = "Organization"
CONSTITUENT_TYPE_HOUSEHOLD = "Household"
CONSTITUENT_TYPE_SYNTHETIC_ID = "-2"  # SourceSystemId for synthetic Individual record

# --- Channel Names (exact match required with Silver Channel.Name) ---
CHANNEL_EMAIL = "Email"
CHANNEL_LETTER = "Letter"
CHANNEL_PHONE_CALL = "Phone Call"

# --- Activity Party Participation Types (from Dynamics 365 activityparty.participationtypemask) ---
# Reference: https://learn.microsoft.com/en-us/dynamics365/customerengagement/on-premises/developer/entities/activityparty
PARTICIPATION_TYPE_FROM = 1       # Sender
PARTICIPATION_TYPE_TO = 2         # Recipient
PARTICIPATION_TYPE_CC = 3         # CC recipient
PARTICIPATION_TYPE_BCC = 4        # BCC recipient
PARTICIPATION_TYPES_SENDER_RECIPIENT = [PARTICIPATION_TYPE_FROM, PARTICIPATION_TYPE_TO]
PARTICIPATION_TYPES_ALL_RECIPIENTS = [PARTICIPATION_TYPE_FROM, PARTICIPATION_TYPE_TO, PARTICIPATION_TYPE_CC, PARTICIPATION_TYPE_BCC]

# --- OpportunityStage State Codes (Dynamics 365 opportunity.statecode optionset) ---
OPPORTUNITY_STATE_OPEN = 0
OPPORTUNITY_STATE_WON = 1
OPPORTUNITY_STATE_LOST = 2
OPPORTUNITY_STAGE_NAME_OPEN = "Open"
OPPORTUNITY_STAGE_NAME_WON = "Won"
OPPORTUNITY_STAGE_NAME_LOST = "Lost"

# --- Account Type (Dynamics 365 account.msnfp_accounttype optionset) ---
ACCOUNT_TYPE_HOUSEHOLD = 844060000  # Household account
ACCOUNT_TYPE_ORGANIZATION = 844060001  # Organization account

# --- OpportunitySalesProcess State Codes (for workflow deduplication) ---
SALES_PROCESS_STATE_ACTIVE = 0  # Active workflow process

# --- Data Constraints ---
MAX_NAME_LENGTH = 500  # Maximum string length for Name fields (varchar constraint)

print("✅ Constants loaded")

# Optionsets (Dimension Tables)

Synchronize Dynamics 365 optionsets to Silver dimension tables using full compare-and-sync (INSERT/UPDATE/DELETE).

**Pattern:** Use single `optionset_sync` instance for all optionset operations.

In [ ]:
# Define the optionsets to sync
# Format: (entity_name, optionset_name, target_table, target_primary_key)
optionsets_to_sync = [
    ("campaign",        "typecode",        "CampaignType",     "CampaignTypeId"),
    ("campaignactivity","channeltypecode", "Channel",          "ChannelId"),
    ("contact",         "gendercode",      "Gender",           "GenderId"),
    ("opportunity",     "statecode",       "OpportunityStage", "OpportunityStageId"),
]

# Run the sync
for entity, optionset, table, pk in optionsets_to_sync:
    try:
        print(f"▶️  {entity}.{optionset} → {table} starting…")
        optionset_sync.sync_optionset(
            entity_name=entity,
            optionset_name=optionset,
            target_table=table,
            target_primary_key=pk,
            # is_global=False,
            # transform_func=None
        )
        print(f"✅ Synchronized {entity}.{optionset} → {table}")
    except Exception as e:
        print(f"❌ {entity}.{optionset} → {table} failed: {e}")
        raise


## ConstituentType - Custom Mapping

In [ ]:
from pyspark.sql import DataFrame
from pyspark.sql.functions import col, lit, lower, trim, when, row_number
from pyspark.sql.window import Window

def transform_constituenttype(df: DataFrame) -> DataFrame:
    """
    Add synthetic 'Individual' type and deduplicate by name (case-insensitive).
    
    WHY: Contacts need ConstituentType but don't have msnfp_accounttype optionset.
         If Bronze contains real 'Individual', prefer it over synthetic.
    """
    synthetic_individual = df.sparkSession.createDataFrame(
        [(CONSTITUENT_TYPE_INDIVIDUAL, CONSTITUENT_TYPE_SYNTHETIC_ID)], 
        ["Name", "SourceSystemId"]
    )
    
    unioned = df.unionByName(synthetic_individual, allowMissingColumns=True)
    
    # Deduplicate: prefer real records over synthetic
    window = Window.partitionBy(lower(trim(col("Name")))).orderBy(
        when(col("SourceSystemId") == CONSTITUENT_TYPE_SYNTHETIC_ID, lit(1)).otherwise(lit(0)),
        col("Name")
    )
    
    return (
        unioned
        .withColumn("rank", row_number().over(window))
        .filter(col("rank") == 1)
        .drop("rank")
    )

optionset_sync.sync_optionset(
    entity_name="",
    optionset_name="msnfp_accounttype",
    target_table="ConstituentType",
    target_primary_key="ConstituentTypeId",
    is_global=True,
    transform_func=transform_constituenttype
)


# Activity Helper Functions

Shared helper functions for activity transformations (Letter, Phonecall, Email).
These are project-specific business logic patterns.

In [ ]:
from pyspark.sql import DataFrame
from pyspark.sql.functions import col, lit, lower, when
from pyspark.sql.types import StringType

def resolve_campaign_for_activity(df: DataFrame) -> DataFrame:
    """
    Resolve Campaign for activity via regardingobjectid.
    
    Business logic: regardingobjectid can point directly to campaign,
    or to campaignactivity which then points to campaign.
    """
    # Read Bronze campaign activity using framework
    campaignactivity = data_sync.reader.read_bronze_table(
        "campaignactivity",
        columns={
            "Id": "ca_Id",
            "regardingobjectid": "ca_regardingobjectid",
            "regardingobjectid_entitytype": "ca_regarding_entity"
        }
    )
    
    # Read Bronze campaign using framework
    campaign_bronze = data_sync.reader.read_bronze_table("campaign", columns={"Id": "c_Id"})
    
    # Join campaign activity (not all activities regard campaignactivity)
    with_ca = df.join(
        campaignactivity,
        (lower(col("regardingobjectid_entitytype")) == lit("campaignactivity")) &
        (col("regardingobjectid") == col("ca_Id")),
        "left"
    )
    
    # Join campaign via campaignactivity
    with_c = with_ca.join(
        campaign_bronze,
        (lower(col("ca_regarding_entity")) == lit("campaign")) &
        (col("ca_regardingobjectid") == col("c_Id")),
        "left"
    )
    
    # Determine Campaign SourceSystemId (direct or via campaignactivity)
    resolved = with_c.withColumn(
        "CampaignSourceSystemId",
        when(lower(col("regardingobjectid_entitytype")) == "campaign", col("regardingobjectid"))
        .when(
            lower(col("regardingobjectid_entitytype")) == "campaignactivity",
            when(lower(col("ca_regarding_entity")) == "campaign", col("c_Id"))
        )
    ).drop("regardingobjectid", "regardingobjectid_entitytype", "ca_Id", "ca_regardingobjectid", "ca_regarding_entity", "c_Id")
    
    # Resolve Campaign SourceSystemId → Silver CampaignId
    campaign_mapping = (
        get_silver_table("SourceSystemIdMapping")
        .filter((col("SourceId") == lit(data_sync.source_id)) & 
                (col("SourceTable") == "Campaign"))
        .select(
            col("SourceSystemId").alias("CampSSId"),
            col("SilverRecordId").alias("CampaignId")
        )
    )
    
    return resolved.join(
        campaign_mapping,
        col("CampaignSourceSystemId").cast(StringType()) == col("CampSSId"),
        "left"
    ).drop("CampaignSourceSystemId", "CampSSId")


def lookup_channel_by_name(channel_name: str) -> DataFrame:
    """Get Channel record by name."""
    return (
        get_silver_table("Channel")
        .filter(lower(trim(col("Name"))) == lit(channel_name.lower()))
        .select("ChannelId")
        .limit(1)
    )


def resolve_activity_party_constituent(df: DataFrame, party_id_col: str, party_type_col: str) -> DataFrame:
    """
    Resolve activity party (Contact/Account) to Constituent (project-specific pattern).
    
    Combines two framework helpers:
    1. Polymorphic lookup: partyid + entitytype → ContactId/AccountId
    2. Constituent resolution: Contact/Account → ConstituentId
    
    Args:
        df: DataFrame with party columns
        party_id_col: Column name for party ID (e.g., "partyid", "PartySrcId")
        party_type_col: Column name for party entity type (e.g., "partyid_entitytype", "PartyType")
    
    Returns:
        DataFrame with ConstituentId column added
    """
    # 1. Resolve polymorphic party (partyid → ContactId/AccountId)
    df_with_party = data_sync.resolve_polymorphic_lookup(
        df,
        lookup_id_column=party_id_col,
        entity_type_column=party_type_col,
        entity_type_mappings={"contact": "Contact", "account": "Account"}
    )
    
    # 2. Resolve Constituent from Contact/Account
    return data_sync.resolve_constituent_id(
        df_with_party,
        contact_fk_column="ContactId",
        account_fk_column="AccountId",
        output_column="ConstituentId"
    )

print("✅ Activity helper functions loaded")

# Data Tables

Synchronize Dynamics 365 data tables with watermark-based incremental sync.

### Transform: Source

In [ ]:
merge_sql = f"""
MERGE INTO {silver_lakehouse_name}.Source AS target
USING (
    SELECT 
        '{source_id}' AS SourceId,
        '{source_name}' AS Name,
        current_timestamp() AS CreatedDate,
        current_timestamp() AS ModifiedDate
) AS source
ON target.SourceId = source.SourceId
WHEN NOT MATCHED THEN INSERT (
    SourceId, CreatedDate, ModifiedDate, Name
) VALUES (
    source.SourceId, source.CreatedDate, source.ModifiedDate, source.Name
)
"""

result = spark.sql(merge_sql)
row = result.collect()[0]

logging.info(f"✅ Rows processed {row['num_affected_rows']}")

### Transform: Country

In [ ]:
from pyspark.sql import functions as F

def transform_country(df):
    """Transform customeraddress.country to Country. Deduplicate by name (case-insensitive)."""
    return (
        D365TransformHelpers.deduplicate_by_window(
            df.filter(F.col("country").isNotNull() & (F.trim(F.col("country")) != "")),
            partition_by=F.upper(F.trim(F.col("country"))),
            order_by="customeraddressid"
        )
        .select(
            F.trim(F.col("country")).alias("Name"),
            F.current_timestamp().alias("CreatedDate"),
            F.current_timestamp().alias("ModifiedDate"),
            F.col("customeraddressid").cast("string").alias("SourceSystemId")
        )
    )

# Custom MERGE SQL - match by name (case-insensitive)
country_merge_sql = """
MERGE INTO {TARGET_TABLE} AS target
USING {SOURCE_VIEW} AS source
ON upper(target.Name) = upper(source.Name)

WHEN NOT MATCHED THEN
    INSERT (
        CountryId, CreatedDate, ModifiedDate, Name, SourceId, SourceSystemId
    )
    VALUES (
        source.CountryId, source.CreatedDate, source.ModifiedDate,
        source.Name, source.SourceId, source.SourceSystemId
    )
"""

data_sync.sync_table(
    source_table="customeraddress",
    source_primary_key="customeraddressid",
    source_columns=["country"],
    target_table="Country",
    target_primary_key="CountryId",
    transform_func=transform_country,
    merge_sql=country_merge_sql
)

### Transform: OpportunityType

In [ ]:
from pyspark.sql import DataFrame
from pyspark.sql.functions import col, current_timestamp

def transform_opportunitytype(df: DataFrame) -> DataFrame:
    """
    Transform opportunitysalesprocess to OpportunityType.
    
    Why workflow join: Dynamics 365 stores workflow process definition in 'workflow' table.
    OpportunityType.Name comes from workflow.name (human-readable label).
    """
    # Get workflow names (WHY: processid is just GUID, we need human-readable name)
    workflow = data_sync.reader.read_bronze_table(
        table_name="workflow",
        columns={
            "workflowid": "wf_id",
            "name": ("wf_name", "string")  # auto-trims
        }
    )
    
    # Join to resolve Name via processid → workflowid
    return (
        df.join(workflow, col("processid") == col("wf_id"), "left")
          .select(
              col("processid"),  # Framework uses this for SourceSystemId
              col("wf_name").alias("Name"),
              current_timestamp().alias("CreatedDate"),
              current_timestamp().alias("ModifiedDate")
          )
    )

data_sync.sync_table(
    source_table="opportunitysalesprocess",
    source_primary_key="processid",
    source_columns=["processid"],
    target_table="OpportunityType",
    target_primary_key="OpportunityTypeId",
    transform_func=transform_opportunitytype
)

### Transform: Campaign

In [ ]:
from pyspark.sql import DataFrame
from pyspark.sql.functions import col
from pyspark.sql.types import TimestampType

def transform_campaign(df: DataFrame) -> DataFrame:
    """Transform campaign to Silver. FK: CampaignType via typecode."""
    return df.select(
        col("Id"),
        col("typecode_SilverRecordId").alias("CampaignTypeId"),
        col("totalactualcost_base").alias("Cost"),
        col("createdon").cast(TimestampType()).alias("CreatedDate"),
        col("actualend").cast(TimestampType()).alias("EndDate"),
        col("modifiedon").cast(TimestampType()).alias("ModifiedDate"),
        col("name").alias("Name"),
        col("actualstart").cast(TimestampType()).alias("StartDate")
    )

data_sync.sync_table(
    source_table="campaign",
    source_primary_key="Id",
    source_columns=[
        "Id", "typecode", "totalactualcost_base",
        "createdon", "actualend", "modifiedon", "name", "actualstart"
    ],
    target_table="Campaign",
    target_primary_key="CampaignId",
    transform_func=transform_campaign,
    fk_mappings={
        "typecode": "CampaignType"
    }
)

### Transform: Address

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.types import TimestampType

# Create in-memory view
# WHY: Addresses exist in both contact and account - union them without writing to disk
spark.sql(f"""
CREATE OR REPLACE TEMPORARY VIEW address_stage_union AS
SELECT 
    address1_city, address1_country, address1_latitude, address1_longitude,
    address1_stateorprovince, address1_postalcode, address1_addressid,
    createdon, modifiedon, IsDelete, SinkModifiedOn
FROM {bronze_lakehouse_name}.contact
WHERE address1_addressid IS NOT NULL
UNION ALL
SELECT 
    address1_city, address1_country, address1_latitude, address1_longitude,
    address1_stateorprovince, address1_postalcode, address1_addressid,
    createdon, modifiedon, IsDelete, SinkModifiedOn
FROM {bronze_lakehouse_name}.account
WHERE address1_addressid IS NOT NULL
""")

print("✅ Temporary view created: address_stage_union (in-memory union)")


def transform_address(df):
    """Transform Address with Region (from customeraddress optionset) and CountryId (from Silver Country)."""
    # Region lookup from customeraddress (WHY: msnfp_region is stored separately)
    ca = data_sync.reader.read_bronze_table(
        table_name="customeraddress",
        columns={"Id": "ca_id", "msnfp_region": "RegionCode"}
    )
    
    # Region optionset labels (msnfp_region code → label)
    # WHY: msnfp_region is a local optionset (integer codes), need human-readable labels
    region_labels = (
        optionset_sync._read_bronze_options(
            entity_name="customeraddress",
            optionset_name="msnfp_region",
            is_global=False
        )
        .select(
            F.col("Option").alias("RegionCode"),
            F.col("Name").alias("Region")
        )
    )
    
    # Country lookup from Silver (source-scoped)
    country = (
        get_silver_table("Country")
        .filter(F.col("SourceId") == F.lit(source_id))
        .select("CountryId", F.col("Name").alias("CountryName"))
    )
    
    # Base selection + Region joins
    base = (
        df.select(
            F.col("address1_city").alias("City"),
            F.col("address1_country").alias("CountryName"),
            F.col("address1_latitude").alias("Latitude"),
            F.col("address1_longitude").alias("Longitude"),
            F.col("address1_stateorprovince").alias("State"),
            F.col("address1_postalcode").alias("ZipCode"),
            F.col("address1_addressid").alias("SourceSystemId"),
            F.col("createdon").cast(TimestampType()).alias("CreatedDate"),
            F.col("modifiedon").cast(TimestampType()).alias("ModifiedDate")
        )
        .join(ca, F.col("SourceSystemId") == F.col("ca_id"), "left")
        .drop("ca_id")
        .join(region_labels, on="RegionCode", how="left")
        .drop("RegionCode")
    )
    
    # Case-insensitive Country join using helper
    result = D365TransformHelpers.join_case_insensitive(
        base, country,
        left_column="CountryName",
        right_column="CountryName",
        how="left"
    ).drop("CountryName")
    
    return result.select(
        F.col("City"),
        F.col("CountryId"),
        F.col("Latitude"),
        F.col("Longitude"),
        F.col("Region"),
        F.col("State"),
        F.col("ZipCode"),
        F.col("CreatedDate"),
        F.col("ModifiedDate"),
        F.col("SourceSystemId")
    )

data_sync.sync_table(
    source_table="address_stage_union",
    source_primary_key="address1_addressid",
    source_columns=[
        "address1_city", "address1_country", "address1_latitude", "address1_longitude",
        "address1_stateorprovince", "address1_postalcode", "createdon", "modifiedon"
    ],
    target_table="Address",
    target_primary_key="AddressId",
    transform_func=transform_address,
    source_table_lakehouse=""
)

### Transform: CampaignChannel

In [ ]:
from pyspark.sql import DataFrame
from pyspark.sql.functions import col, lit, lower, expr
from pyspark.sql.types import StringType

def _load_channel_metadata() -> DataFrame:
    """
    Load optionset metadata mapping channeltypecode to human-readable labels.
    
    WHY: Dynamics 365 stores campaignactivity.channeltypecode as integer option values.
         We need the localized label (e.g., "Email", "Phone Call", "Letter") 
         to match against Channel.Name dimension table.
    
    Business context: channeltypecode is NOT the same as Campaign.typecode!
         - Campaign.typecode = Type of campaign (e.g., "Email Campaign")
         - campaignactivity.channeltypecode = Channel used in activity (e.g., "Email")
    
    Framework usage: Uses optionset_sync._read_bronze_options() for consistent
         optionset handling (language selection, null filtering, deduplication).
    
    Returns:
        DataFrame with columns: typecode_opt (int), OptChannelName (string)
    """
    return (
        optionset_sync._read_bronze_options(
            entity_name="campaignactivity",
            optionset_name="channeltypecode",
            is_global=False
        )
        .select(
            col("Option").cast("bigint").alias("typecode_opt"),
            col("Name").alias("OptChannelName")
        )
    )


def _filter_campaign_activities(df: DataFrame) -> DataFrame:
    """
    Filter to only activities where regardingobjectid points to a campaign.
    
    WHY: campaignactivity records can have regardingobjectid pointing to different
         entity types (campaign, lead, opportunity, etc.). We only want activities
         that are explicitly linked to campaigns (regardingobjectid_entitytype = "campaign").
    
    Business rule: Activity must have:
         - regardingobjectid pointing to a campaign (not lead/opportunity/etc.)
         - channeltypecode indicating which channel was used
    
    Returns:
        DataFrame with columns: ActivityId, CampaignSrcId, ChannelTypeCode
    """
    return (
        df.filter(lower(col("regardingobjectid_entitytype")) == lit("campaign"))
          .select(
              col("activityid").cast(StringType()).alias("ActivityId"),
              col("regardingobjectid").cast(StringType()).alias("CampaignSrcId"),
              col("channeltypecode").cast("bigint").alias("ChannelTypeCode")
          )
          .filter(col("CampaignSrcId").isNotNull() & col("ActivityId").isNotNull())
    )


def _enrich_channel_names(df: DataFrame) -> DataFrame:
    """
    Add human-readable channel names to activities via optionset lookup.
    
    WHY: channeltypecode is just an integer (e.g., 1, 2, 3).
         We need the label (e.g., "Email", "Phone Call") to match against
         Channel.Name in the dimension table.
    
    Business logic: Maps channeltypecode → LocalizedLabel from OptionsetMetadata
    
    Returns:
        Original DataFrame + OptChannelName column
    """
    optionset = _load_channel_metadata()
    
    return df.join(
        optionset,
        col("ChannelTypeCode") == col("typecode_opt"),
        "left"
    ).select(
        "ActivityId", "CampaignSrcId", "ChannelTypeCode", "OptChannelName"
    )


def _resolve_campaign_ids(df: DataFrame) -> DataFrame:
    """
    Translate Bronze campaign IDs to Silver CampaignId GUIDs.
    
    WHY: campaignactivity.regardingobjectid contains Bronze campaign IDs (source-specific).
         We need Silver CampaignId GUIDs to create foreign key relationship.
    
    Uses: Silver Campaign table filtered by SourceId (this data source only)
    Lookup: CampaignSrcId (Bronze) → CampaignId (Silver GUID)
    
    Returns:
        Original DataFrame + CampaignId column
    """
    campaigns = (
        get_silver_table("Campaign")
        .filter(col("SourceId") == lit(data_sync.source_id))
        .select(
            col("SourceSystemId").cast(StringType()).alias("CampSSId"),
            col("CampaignId")
        )
    )
    
    return df.join(
        campaigns,
        col("CampaignSrcId") == col("CampSSId"),
        "left"
    ).select(
        "ActivityId", "CampaignSrcId", "ChannelTypeCode", "OptChannelName", "CampaignId"
    )


def _resolve_channel_ids(df: DataFrame) -> DataFrame:
    """
    Map channel name labels to ChannelId dimension keys.
    
    WHY: Channel is a shared dimension table (synced from channeltypecode optionset).
         We need to match OptChannelName → Channel.Name → ChannelId.
    
    Business context: Channel dimension is shared across all data sources.
         Names like "Email", "Phone Call", "Letter" are standardized.
    
    Returns:
        Original DataFrame + ChannelId column
    """
    channels = get_silver_table("Channel").select(
        col("Name").alias("ChannelName"),
        col("ChannelId")
    )
    
    return df.join(
        channels,
        col("OptChannelName") == col("ChannelName"),
        "left"
    ).select(
        "ActivityId", "CampaignSrcId", "ChannelTypeCode", "CampaignId", "ChannelId"
    )


def _build_campaign_channel_pairs(df: DataFrame) -> DataFrame:
    """
    Orchestrate link table creation: filter → enrich → resolve → deduplicate.
    
    WHY: CampaignChannel is a many-to-many link table (Campaign ↔ Channel).
         Multiple activities can create the same (CampaignId, ChannelId) pair.
         We need exactly ONE link record per unique pair.
    
    Business logic flow:
         1. Filter to campaign-related activities
         2. Enrich with channel names (optionset lookup)
         3. Resolve Campaign IDs (Bronze → Silver)
         4. Resolve Channel IDs (name → dimension)
         5. Deduplicate by (CampaignId, ChannelId) pair
    
    SourceSystemId strategy: Keep ActivityId for traceability
         (tracks which activity created the pair, useful for debugging)
    
    Returns:
        DataFrame with columns: CampaignId, ChannelId, ActivityId
    """
    # Step 1: Filter to campaign activities
    filtered = _filter_campaign_activities(df)
    
    # Step 2: Enrich with channel names
    with_names = _enrich_channel_names(filtered)
    
    # Step 3: Resolve Campaign IDs
    with_campaign = _resolve_campaign_ids(with_names)
    
    # Step 4: Resolve Channel IDs
    with_channel = _resolve_channel_ids(with_campaign)
    
    # Step 5: Filter out incomplete resolutions and deduplicate
    return (
        with_channel
        .filter(col("CampaignId").isNotNull() & col("ChannelId").isNotNull())
        .dropDuplicates(["CampaignId", "ChannelId", "ActivityId"])
    )


def transform_campaign_channel(df: DataFrame) -> DataFrame:
    """
    Transform campaignactivity → CampaignChannel link table.
    
    Source: campaignactivity (Bronze table tracking activities performed for campaigns)
    Target: CampaignChannel (Silver link table: Campaign ↔ Channel many-to-many)
    
    Business logic:
         Each campaign activity has a channeltypecode indicating which channel was used
         (e.g., Email, Phone, Letter). This creates a relationship between the campaign
         and that channel. Multiple activities can use the same channel for the same
         campaign → deduplicate to one link record per (Campaign, Channel) pair.
    
    WHY campaignactivity (not Campaign)?
         - Campaign.typecode = campaign TYPE (e.g., "Email Campaign")
         - campaignactivity.channeltypecode = actual CHANNEL used (e.g., "Email")
         - A "Multi-Channel Campaign" might use Email, Phone, and Letter channels
         - We derive this from actual activities, not from campaign metadata
    
    SourceSystemId = ActivityId:
         - Tracks which activity created the pair (traceability)
         - Framework uses this for SourceSystemIdMapping tracking
         - Multiple activities can create same pair → dedupe keeps one ActivityId
    """
    pairs = _build_campaign_channel_pairs(df)
    
    return (
        pairs
        .withColumn("CampaignChannelId", expr("uuid()"))
        .withColumn("SourceSystemId", col("ActivityId"))
        .select("CampaignChannelId", "ChannelId", "CampaignId", "SourceSystemId")
    )


merge_sql_campaign_channel = f"""
MERGE INTO {data_sync.target_lakehouse}.CampaignChannel AS target
USING {{SOURCE_VIEW}} AS source
  ON  target.CampaignId = source.CampaignId
 AND  target.ChannelId  = source.ChannelId
WHEN NOT MATCHED THEN INSERT (CampaignChannelId, ChannelId, CampaignId)
VALUES (source.CampaignChannelId, source.ChannelId, source.CampaignId)
"""


def _cleanup_campaign_channel_links():
    """
    Remove stale (CampaignId, ChannelId) pairs that no longer exist in Bronze.
    
    WHY cleanup is necessary for derived link tables:
         CampaignChannel is DERIVED from campaignactivity (not a primary entity).
         Framework MERGE matches on composite key (CampaignId, ChannelId), but tracks
         individual ActivityId via SourceSystemIdMapping.
    
    Problem scenarios requiring cleanup:
         1. Activity changes channeltypecode: Email → Phone
            - Framework inserts new pair (Campaign, Phone)
            - Old pair (Campaign, Email) remains stale if no other activities use Email
         
         2. Activity deleted (IsDelete=1)
            - Framework deletes record with that ActivityId
            - But if multiple activities created same pair, pair should remain!
            - Cleanup determines if pair is TRULY stale (no activities support it)
         
         3. Activity changes regardingobjectid: Campaign1 → Campaign2
            - Creates pair for Campaign2
            - Pair for Campaign1 becomes stale if no other activities support it
    
    Pattern: left_anti join
         expected_pairs = rebuild from current Bronze state
         stale_pairs = (existing Silver pairs) - (expected pairs)
         DELETE stale_pairs
    
    NOTE: This pattern is needed because:
         - Multiple ActivityIds can produce the same (CampaignId, ChannelId) pair
         - Framework SourceSystemIdMapping tracks per-ActivityId, not per-pair
         - Composite key MERGE doesn't automatically clean up stale pairs
    """
    # Rebuild expected pairs from current Bronze state
    expected = _build_campaign_channel_pairs(
        data_sync.reader.read_bronze_table(
            "campaignactivity",
            columns=[
                "activityid",
                "regardingobjectid",
                "regardingobjectid_entitytype",
                "channeltypecode"
            ]
        )
    ).select("CampaignId", "ChannelId").dropDuplicates()
    
    # Get campaigns owned by this source
    my_campaigns = (
        get_silver_table("Campaign")
        .filter(col("SourceId") == lit(data_sync.source_id))
        .select("CampaignId")
    )
    
    # Get existing pairs in Silver for this source's campaigns
    target_pairs = (
        get_silver_table("CampaignChannel").alias("t")
        .join(my_campaigns.alias("c"), col("t.CampaignId") == col("c.CampaignId"), "inner")
        .select("t.CampaignId", "t.ChannelId")
    )
    
    # Find stale pairs: in target but not in expected
    stale = (
        target_pairs.alias("t")
        .join(
            expected.alias("e"),
            (col("t.CampaignId") == col("e.CampaignId")) &
            (col("t.ChannelId") == col("e.ChannelId")),
            "left_anti"
        )
    )
    
    # Delete stale pairs
    stale.createOrReplaceTempView("_stale_campaign_channel")
    
    spark.sql(f"""
        MERGE INTO {data_sync.target_lakehouse}.CampaignChannel AS t
        USING _stale_campaign_channel AS s
          ON  t.CampaignId = s.CampaignId
         AND  t.ChannelId  = s.ChannelId
        WHEN MATCHED THEN DELETE
    """)
    
    spark.catalog.dropTempView("_stale_campaign_channel")


data_sync.sync_table(
    source_table="campaignactivity",
    source_primary_key="activityid",
    source_columns=["activityid", "regardingobjectid", "regardingobjectid_entitytype", "channeltypecode"],
    target_table="CampaignChannel",
    target_primary_key="CampaignChannelId",
    transform_func=transform_campaign_channel,
    merge_sql=merge_sql_campaign_channel
)

_cleanup_campaign_channel_links()

### Transform: EmailEngagement

In [ ]:
from pyspark.sql import DataFrame
from pyspark.sql.functions import col, lit
from pyspark.sql.types import StringType, TimestampType

def transform_emailengagement(df: DataFrame) -> DataFrame:
    """
    Transform email → EmailEngagement.
    
    WHY: Track email campaign activities. Resolve Campaign via regardingobjectid 
         (direct link to campaign OR indirect via campaignactivity → campaign).
         Email channel assignment for all email activities.
    """
    # Base email attributes with regardingobjectid for campaign resolution
    base = df.select(
        col("Id").alias("EmailId"),
        col("createdon").cast(TimestampType()).alias("CreatedDate"),
        col("modifiedon").cast(TimestampType()).alias("ModifiedDate"),
        col("senton").cast(TimestampType()).alias("SendDate"),
        col("subject").alias("Subject"),
        col("versionnumber").alias("VariantType"),
        col("regardingobjectid"),
        col("regardingobjectid_entitytype")
    )
    
    # Resolve Campaign using shared helper (handles direct + indirect resolution)
    with_campaign = resolve_campaign_for_activity(base)
    
    # Attach Email channel using shared helper
    email_channel = lookup_channel_by_name(CHANNEL_EMAIL)
    with_channel = with_campaign.crossJoin(email_channel)
    
    return (
        with_channel.select(
            col("EmailId").alias("Id"),  # Framework expects "Id" for SourceSystemId
            col("CampaignId"),
            col("CreatedDate"),
            col("EmailId"),
            col("ModifiedDate"),
            col("SendDate"),
            col("Subject"),
            col("VariantType"),
            col("ChannelId")
        )
        .dropDuplicates(["Id"])
    )

data_sync.sync_table(
    source_table="email",
    source_primary_key="Id",
    source_columns=[
        "Id",
        "createdon", "modifiedon", "senton",
        "subject", "versionnumber",
        "regardingobjectid", "regardingobjectid_entitytype",
        "SinkModifiedOn"
    ],
    target_table="EmailEngagement",
    target_primary_key="EmailEngagementId",
    transform_func=transform_emailengagement
)

### Transform: Account

In [ ]:
from pyspark.sql import DataFrame
from pyspark.sql.functions import col
from pyspark.sql.types import TimestampType

def transform_account(df: DataFrame) -> DataFrame:
    """Transform account to Silver. FK: Address via address1_addressid."""
    return df.select(
        col("Id"),
        col("address1_addressid_SilverRecordId").alias("AddressId"),
        col("name").alias("Name"),
        col("emailaddress1").alias("Email"),
        col("createdon").cast(TimestampType()).alias("CreatedDate"),
        col("modifiedon").cast(TimestampType()).alias("ModifiedDate")
    )

data_sync.sync_table(
    source_table="account",
    source_primary_key="Id",
    source_columns=[
        "Id", "name", "emailaddress1", "address1_addressid",
        "createdon", "modifiedon"
    ],
    target_table="Account",
    target_primary_key="AccountId",
    transform_func=transform_account,
    fk_mappings={
        "address1_addressid": "Address"
    }
)

### Transform: Contact

In [ ]:
from pyspark.sql import DataFrame
from pyspark.sql.functions import col, lit
from pyspark.sql.types import TimestampType

def transform_contact(df: DataFrame) -> DataFrame:
    """Transform contact to Silver. FKs: Gender, Address."""
    return df.select(
        col("Id"),
        col("gendercode_SilverRecordId").alias("GenderId"),
        col("address1_addressid_SilverRecordId").alias("AddressId"),
        col("firstname").alias("FirstName"),
        col("lastname").alias("LastName"),
        col("emailaddress1").alias("Email"),
        col("birthdate").cast("date").alias("BirthDate"),
        col("createdon").cast(TimestampType()).alias("CreatedDate"),
        col("modifiedon").cast(TimestampType()).alias("ModifiedDate")
    )

data_sync.sync_table(
    source_table="contact",
    source_primary_key="Id",
    source_columns=[
        "Id", "firstname", "lastname", "emailaddress1", "birthdate",
        "gendercode", "address1_addressid", "createdon", "modifiedon"
    ],
    target_table="Contact",
    target_primary_key="ContactId",
    transform_func=transform_contact,
    fk_mappings={
        "gendercode": "Gender",
        "address1_addressid": "Address"
    }
)

### Transform: Constituent

In [ ]:
from pyspark.sql import DataFrame
from pyspark.sql.functions import col, lit, when

def transform_constituent_from_contact(df: DataFrame) -> DataFrame:
    """Transform contact to Constituent (Individual type)."""
    individual_type = (
        get_silver_table("ConstituentType")
        .filter(col("Name") == CONSTITUENT_TYPE_INDIVIDUAL)
        .select("ConstituentTypeId")
        .first()
    )

    if not individual_type:
        raise ValueError(f"ConstituentType '{CONSTITUENT_TYPE_INDIVIDUAL}' not found")

    contact_mapping = (
        get_silver_table("SourceSystemIdMapping")
        .filter((col("SourceId") == lit(source_id)) & 
                (col("SourceTable") == "Contact"))
        .select(
            col("SourceSystemId").alias("ContactSourceId"),
            col("SilverRecordId").alias("ContactId")
        )
    )

    return (
        df.join(contact_mapping, col("Id") == col("ContactSourceId"), "inner")
          .select(
              col("Id"),
              col("ContactId"),
              lit(None).cast("string").alias("AccountId"),
              lit(individual_type["ConstituentTypeId"]).alias("ConstituentTypeId")
          )
    )

# Custom MERGE SQL - matches on (ContactId, AccountId) composite key with NULL-safe comparison
constituent_merge_sql = """
MERGE INTO {TARGET_TABLE} AS target
USING {SOURCE_VIEW} AS source
ON target.ContactId <=> source.ContactId
   AND target.AccountId <=> source.AccountId
WHEN MATCHED THEN
    UPDATE SET target.ConstituentTypeId = source.ConstituentTypeId
WHEN NOT MATCHED THEN
    INSERT (ConstituentId, ContactId, AccountId, ConstituentTypeId)
    VALUES (source.ConstituentId, source.ContactId, source.AccountId, source.ConstituentTypeId)
"""

data_sync.sync_table(
    source_table="contact",
    source_primary_key="Id",
    source_columns=["Id"],
    target_table="Constituent",
    target_primary_key="ConstituentId",
    transform_func=transform_constituent_from_contact,
    merge_sql=constituent_merge_sql
)

def transform_constituent_from_account(df: DataFrame) -> DataFrame:
    """
    Transform account to Constituent (Org/Household type based on msnfp_accounttype).
    """
    constituent_types = (
        get_silver_table("ConstituentType")
        .select("Name", "ConstituentTypeId")
    )

    type_map = {r["Name"]: r["ConstituentTypeId"] for r in constituent_types.collect()}
    org_type_id = type_map.get(CONSTITUENT_TYPE_ORGANIZATION)
    household_type_id = type_map.get(CONSTITUENT_TYPE_HOUSEHOLD)

    if not org_type_id or not household_type_id:
        raise ValueError(f"ConstituentType '{CONSTITUENT_TYPE_ORGANIZATION}' or '{CONSTITUENT_TYPE_HOUSEHOLD}' not found")

    account_mapping = (
        get_silver_table("SourceSystemIdMapping")
        .filter((col("SourceId") == lit(source_id)) & 
                (col("SourceTable") == "Account"))
        .select(
            col("SourceSystemId").alias("AccountSourceId"),
            col("SilverRecordId").alias("AccountId")
        )
    )

    return (
        df.join(account_mapping, col("Id") == col("AccountSourceId"), "inner")
          .select(
              col("Id"),
              lit(None).cast("string").alias("ContactId"),
              col("AccountId"),
              when(col("msnfp_accounttype") == ACCOUNT_TYPE_ORGANIZATION, lit(org_type_id))
              .when(col("msnfp_accounttype") == ACCOUNT_TYPE_HOUSEHOLD, lit(household_type_id))
              .alias("ConstituentTypeId")
          )
          .filter(col("ConstituentTypeId").isNotNull())
    )

data_sync.sync_table(
    source_table="account",
    source_primary_key="Id",
    source_columns=["Id", "msnfp_accounttype"],
    target_table="Constituent",
    target_primary_key="ConstituentId",
    transform_func=transform_constituent_from_account,
    merge_sql=constituent_merge_sql
)

### Transform: ConstituentEmailEngagement

In [ ]:
from pyspark.sql import DataFrame
from pyspark.sql.functions import col, expr, lit, lower
from pyspark.sql.types import StringType, TimestampType, BooleanType

def _resolve_email_engagement_link(df: DataFrame) -> DataFrame:
    """
    Link ConstituentEmailEngagement to parent EmailEngagement record.
    
    WHY: EmailEngagement (email activity) is parent; ConstituentEmailEngagement tracks per-recipient.
         Need EmailEngagementId (Silver GUID) and EmailId (from parent EmailEngagement).
    """
    mapping = (
        get_silver_table("SourceSystemIdMapping")
        .filter(col("SourceId") == lit(data_sync.source_id))
        .filter(col("SourceTable") == lit("EmailEngagement"))
        .select(
            col("SourceSystemId").cast(StringType()).alias("_EmailActivityId"),
            col("SilverRecordId").alias("EmailEngagementId")
        )
    )
    
    email_engagement = (
        get_silver_table("EmailEngagement")
        .select(col("EmailEngagementId"), col("EmailId"))
    )
    
    return (
        df.join(mapping, df["EmailActivityId"] == col("_EmailActivityId"), "left")
        .drop("_EmailActivityId")
        .join(email_engagement, on="EmailEngagementId", how="left")
    )

def transform_constituent_email_engagement(df: DataFrame) -> DataFrame:
    """
    Transform email activity → ConstituentEmailEngagement (per-recipient engagement tracking).
    
    WHY: One email can have multiple recipients (TO/CC/BCC). Track engagement per recipient.
         SourceSystemId = ActivityPartyId (unique per email-person-role combination).
         addressused field captures actual email address used (may differ from contact's primary).
    """
    # Extract email engagement metrics
    email_base = df.select(
        col("Id").alias("EmailActivityId"),
        col("createdon").cast(TimestampType()).alias("CreatedDate"),
        col("modifiedon").cast(TimestampType()).alias("ModifiedDate"),
        col("senton").cast(TimestampType()).alias("SendDate"),
        col("subject").alias("EmailSubject"),
        (col("lastopenedtime").isNotNull()).cast(BooleanType()).alias("WasOpened"),
        col("lastopenedtime").cast(TimestampType()).alias("OpenedDate"),
        (col("linksclickedcount") > 0).cast(BooleanType()).alias("ClickThrough"),
        expr("CASE WHEN linksclickedcount > 0 THEN cast(modifiedon as timestamp) ELSE NULL END").alias("ClickThroughDate")
    )
    
    # Load activityparty with addressused (framework helper doesn't include addressused)
    activityparty = data_sync.reader.read_bronze_table(
        table_name="activityparty",
        columns={
            "activityid": "EmailActivityId",
            "partyid": "partyid",
            "partyid_entitytype": "partyid_entitytype",
            "Id": "SourceSystemId",
            "addressused": "ConstituentEmail"
        },
        filters=[
            col("participationtypemask").isin(*PARTICIPATION_TYPES_ALL_RECIPIENTS),
            col("partyid").isNotNull(),
            lower(col("partyid_entitytype")).isin("contact", "account")
        ]
    ).withColumn("partyid_entitytype", lower(col("partyid_entitytype"))) \
     .dropDuplicates(["EmailActivityId", "partyid", "partyid_entitytype", "SourceSystemId"])
    
    exploded = email_base.join(activityparty, on="EmailActivityId", how="inner")
    
    # Resolve partyid (contact/account) → ConstituentId
    resolved = data_sync.resolve_polymorphic_lookup(
        exploded,
        lookup_id_column="partyid",
        entity_type_column="partyid_entitytype",
        entity_type_mappings={"contact": "Contact", "account": "Account"}
    )
    
    with_constituent = data_sync.resolve_constituent_id(
        resolved,
        contact_fk_column="ContactId",
        account_fk_column="AccountId",
        output_column="ConstituentId"
    )
    
    linked = _resolve_email_engagement_link(with_constituent)
    
    return (
        linked
        .filter(col("ConstituentId").isNotNull())
        .select(
            col("EmailEngagementId"),
            lit(None).cast(StringType()).alias("Timezone"),
            col("ConstituentEmail"),
            col("ConstituentId"),
            col("CreatedDate"),
            col("EmailId"),
            col("EmailSubject"),
            col("ModifiedDate"),
            col("OpenedDate"),
            col("SendDate"),
            col("SourceSystemId"),
            col("WasOpened"),
            col("ClickThrough"),
            col("ClickThroughDate")
        )
        .dropDuplicates(["SourceSystemId"])
    )

data_sync.sync_table(
    source_table="email",
    source_primary_key="Id",
    source_columns=[
        "Id",
        "createdon", "modifiedon", "senton",
        "subject",
        "lastopenedtime", "linksclickedcount",
        "SinkModifiedOn"
    ],
    target_table="ConstituentEmailEngagement",
    target_primary_key="ConstituentEmailEngagementId",
    transform_func=transform_constituent_email_engagement
)

### Transform: ConstituentOpportunityStage

In [ ]:
from pyspark.sql import DataFrame
from pyspark.sql.functions import col, lit, lower, sha2, concat_ws
from pyspark.sql.types import StringType

def _load_opportunity_stage_mapping() -> DataFrame:
    """
    Load OpportunityStage mapping (statecode → OpportunityStageId).
    
    WHY: Dynamics 365 stores opportunity state as numeric code (0, 1, 2).
         OpportunityStage dimension table was synced first with these codes.
         Sticky mapping ensures consistent OpportunityStageId across runs.
    
    Returns:
        DataFrame with columns: StateCodeStr, OpportunityStageId
    """
    return (
        get_silver_table("SourceSystemIdMapping")
        .filter((col("SourceId") == lit(data_sync.source_id)) &
                (col("SourceTable") == "OpportunityStage"))
        .select(
            col("SourceSystemId").alias("StateCodeStr"),
            col("SilverRecordId").alias("OpportunityStageId")
        )
    )

def _prepare_opportunity_base_data(df: DataFrame) -> DataFrame:
    """
    Extract and normalize opportunity columns needed for constituent-stage pairing.
    
    WHY: We need:
        - statecode: To resolve which OpportunityStage
        - customerid + entitytype: To resolve which Constituent (polymorphic lookup)
    """
    return df.select(
        col("Id").alias("OpportunitySourceId"),
        col("customerid").alias("customerid"),
        lower(col("customerid_entitytype")).alias("customerid_entitytype"),
        col("statecode").alias("StateCodeStr")
    )

def _resolve_opportunity_stage(base_df: DataFrame, stage_mapping: DataFrame) -> DataFrame:
    """
    Join opportunity data with OpportunityStage mapping.
    
    WHY: Converts numeric statecode to stable OpportunityStageId GUID.
         Uses sticky mapping to preserve IDs across sync runs.
    """
    return base_df.join(stage_mapping, on="StateCodeStr", how="left")

def _resolve_opportunity_customer_constituent(df: DataFrame) -> DataFrame:
    """
    Resolve opportunity customer (polymorphic Contact/Account) to Constituent.
    
    Business Logic:
    - opportunity.customerid can point to Contact OR Account (indicated by customerid_entitytype)
    - Contact/Account both link to Constituent
    - Result: ConstituentId who owns this opportunity
    
    WHY Two-step resolution:
    1. Polymorphic lookup: customerid → ContactId/AccountId (based on entitytype)
    2. Constituent resolution: Contact/Account → ConstituentId
    """
    # Step 1: Resolve polymorphic customerid → ContactId/AccountId
    with_party = data_sync.resolve_polymorphic_lookup(
        df,
        lookup_id_column="customerid",
        entity_type_column="customerid_entitytype",
        entity_type_mappings={"contact": "Contact", "account": "Account"}
    )
    
    # Step 2: Resolve Contact/Account → Constituent
    return data_sync.resolve_constituent_id(
        with_party,
        contact_fk_column="ContactId",
        account_fk_column="AccountId",
        output_column="ConstituentId"
    )

def _build_constituent_opportunity_stage_pairs(df: DataFrame) -> DataFrame:
    """
    Build final ConstituentOpportunityStage link table with deterministic composite key.
    
    WHY Deterministic Hash:
    - Link table tracks many-to-many: Constituent ↔ OpportunityStage
    - Same (ConstituentId, OpportunityStageId) pair must get same SourceSystemId
    - SHA2 hash ensures idempotence: re-running sync won't create duplicate pairs
    - Framework uses SourceSystemId for MERGE matching via SourceSystemIdMapping
    
    WHY Filter NULLs:
    - Foreign key constraints require valid ConstituentId and OpportunityStageId
    - NULL values indicate resolution failure (missing Contact/Account or invalid statecode)
    """
    return (
        df.where(col("ConstituentId").isNotNull() & col("OpportunityStageId").isNotNull())
          .select(
              # Deterministic composite key: hash of (ConstituentId, OpportunityStageId)
              sha2(concat_ws("|", col("ConstituentId"), col("OpportunityStageId")), 256)
                  .alias("SourceSystemId"),
              col("ConstituentId"),
              col("OpportunityStageId")
          )
          .dropDuplicates(["SourceSystemId"])
    )

def transform_constituent_opportunity_stage(df: DataFrame) -> DataFrame:
    """
    Transform opportunity to ConstituentOpportunityStage link table.
    
    Business Logic:
    ConstituentOpportunityStage tracks which constituents have opportunities in which stages.
    This is a many-to-many link table (one constituent can have multiple opportunities,
    and opportunities can be in different stages over time).
    
    Data Flow:
    opportunity.statecode → OpportunityStageId (via sticky mapping)
    opportunity.customerid → ConstituentId (via polymorphic Contact/Account lookup)
    → Link table pair: (ConstituentId, OpportunityStageId)
    
    WHY This Pattern:
    - Enables analytics: "Which constituents have opportunities in Won stage?"
    - Deterministic hash prevents duplicate pairs across sync runs
    - Framework handles MERGE automatically using hash-based SourceSystemId
    """
    # Step 1: Load OpportunityStage mapping (statecode → OpportunityStageId)
    stage_mapping = _load_opportunity_stage_mapping()
    
    # Step 2: Prepare opportunity base data
    base_data = _prepare_opportunity_base_data(df)
    
    # Step 3: Resolve OpportunityStage from statecode
    with_stage = _resolve_opportunity_stage(base_data, stage_mapping)
    
    # Step 4: Resolve Constituent from polymorphic customerid
    with_constituent = _resolve_opportunity_customer_constituent(with_stage)
    
    # Step 5: Build final link table pairs with deterministic hash
    return _build_constituent_opportunity_stage_pairs(with_constituent)

data_sync.sync_table(
    source_table="opportunity",
    source_primary_key="Id",
    source_columns=["Id", "customerid", "customerid_entitytype", "statecode", "SinkModifiedOn"],
    target_table="ConstituentOpportunityStage",
    target_primary_key="ConstituentOpportunityStageId",
    transform_func=transform_constituent_opportunity_stage
)

### Transform: Letter

In [ ]:
from pyspark.sql import DataFrame
from pyspark.sql.functions import col, lit
from pyspark.sql.types import StringType, TimestampType

def transform_letter(df: DataFrame) -> DataFrame:
    """
    Transform letter to Silver via activityparty explosion (1 letter → N recipients).
    Resolves CampaignId and ConstituentId using Activity Helper Functions.
    """
    # Read activity parties (1:N explosion point - one letter to multiple recipients)
    activityparty = data_sync.reader.read_activity_parties(
        activity_id_column="activityid",
        participation_types=PARTICIPATION_TYPES_SENDER_RECIPIENT
    )
    
    # Join letter with parties
    with_parties = (
        df.select(
            col("Id").alias("activityid"),
            col("createdon").cast(TimestampType()).alias("CreatedDate"),
            col("modifiedon").cast(TimestampType()).alias("ModifiedDate"),
            col("actualend").cast(TimestampType()).alias("SentDate"),
            col("subject").alias("Subject"),
            col("regardingobjectid"),
            col("regardingobjectid_entitytype")
        )
        .join(activityparty, on="activityid", how="inner")
    )
    
    # Resolve Campaign and Constituent using shared Activity helpers
    with_campaign = resolve_campaign_for_activity(with_parties)
    with_constituent = resolve_activity_party_constituent(
        with_campaign,
        party_id_col="partyid",
        party_type_col="partyid_entitytype"
    )
    
    # Add Letter channel
    channel_letter = lookup_channel_by_name(CHANNEL_LETTER)
    
    return (
        with_constituent.crossJoin(channel_letter)
        .select(
            col("ActivityPartyId").alias("SourceSystemId"),
            "CampaignId",
            "ConstituentId",
            "ChannelId",
            "CreatedDate",
            "ModifiedDate",
            "SentDate",
            "Subject"
        )
        .filter(col("ConstituentId").isNotNull())
    )

# WHY composite key: Letter can have same ActivityPartyId from different sources,
# and same constituent can receive multiple letters
letter_merge_sql = """
MERGE INTO {TARGET_TABLE} AS target
USING {SOURCE_VIEW} AS source
  ON  target.SourceId = source.SourceId
  AND target.SourceSystemId = source.SourceSystemId
  AND target.ConstituentId = source.ConstituentId

WHEN MATCHED THEN UPDATE SET
    target.ModifiedDate = source.ModifiedDate,
    target.SentDate = source.SentDate,
    target.Subject = source.Subject,
    target.CampaignId = source.CampaignId,
    target.ChannelId = source.ChannelId

WHEN NOT MATCHED THEN INSERT (
    LetterId, CampaignId, ChannelId, ConstituentId, CreatedDate,
    ModifiedDate, SentDate, Subject, SourceId, SourceSystemId
) VALUES (
    source.LetterId, source.CampaignId, source.ChannelId, source.ConstituentId,
    source.CreatedDate, source.ModifiedDate, source.SentDate, source.Subject,
    source.SourceId, source.SourceSystemId
)
"""

data_sync.sync_activity_table(
    source_table="letter",
    source_primary_key="Id",
    source_columns=[
        "Id", "createdon", "modifiedon", "actualend", "subject",
        "regardingobjectid", "regardingobjectid_entitytype"
    ],
    target_table="Letter",
    target_primary_key="LetterId",
    activity_party_join_func=transform_letter,
    merge_sql=letter_merge_sql
)

### Transform: Opportunity

In [ ]:
from pyspark.sql import DataFrame, Window
from pyspark.sql import functions as F
from pyspark.sql.functions import col, lit, trim, substring, when, row_number
from pyspark.sql.types import TimestampType, DecimalType, StringType

def resolve_opportunity_type(df: DataFrame) -> DataFrame:
    """
    Resolve OpportunityType via opportunitysalesprocess.
    
    Business Logic:
    - One opportunity can have multiple sales processes (workflows)
    - WHY Window deduplication: Prefer active process (statecode=0), then most recent
    - OpportunityType comes from the chosen process's processid
    """
    # Read opportunitysalesprocess using framework (auto-trims strings)
    sales_process = data_sync.reader.read_bronze_table(
        "opportunitysalesprocess",
        columns={
            "opportunityid": ("OpportunitySalesProcessOpportunityId", "string"),
            "processid": ("OpportunitySalesProcessProcessId", "string"),
            "statecode": "OpportunitySalesProcessStateCode",
            "SinkModifiedOn": ("OpportunitySalesProcessModified", "timestamp")
        }
    )
    
    # Deduplicate: prefer active processes, then latest modified
    window = Window.partitionBy("OpportunitySalesProcessOpportunityId").orderBy(
        when(col("OpportunitySalesProcessStateCode") == SALES_PROCESS_STATE_ACTIVE, lit(0)).otherwise(lit(1)),
        col("OpportunitySalesProcessModified").desc_nulls_last()
    )
    
    active_process = (
        sales_process
        .withColumn("rank", row_number().over(window))
        .filter(col("rank") == 1)
        .select("OpportunitySalesProcessOpportunityId", "OpportunitySalesProcessProcessId")
    )
    
    # Join opportunity → sales process
    with_process = df.join(
        active_process,
        col("Id").cast(StringType()) == col("OpportunitySalesProcessOpportunityId"),
        "left"
    )
    
    # Resolve processid → OpportunityTypeId via Silver OpportunityType (source-scoped)
    opportunity_type = (
        get_silver_table("OpportunityType")
        .filter(col("SourceId") == lit(data_sync.source_id))
        .select(
            col("SourceSystemId").alias("ProcessIdString"),
            col("OpportunityTypeId")
        )
    )
    
    return with_process.join(
        opportunity_type,
        col("OpportunitySalesProcessProcessId") == col("ProcessIdString"),
        "left"
    ).drop("OpportunitySalesProcessOpportunityId", "OpportunitySalesProcessProcessId", "ProcessIdString")


def resolve_opportunity_stage(df: DataFrame) -> DataFrame:
    """
    Resolve OpportunityStage via statecode mapping.
    
    Maps Dynamics 365 statecode values to OpportunityStage names:
    - 0 → Open, 1 → Won, 2 → Lost
    """
    opportunity_stage = (
        get_silver_table("OpportunityStage")
        .select(
            col("Name").alias("OpportunityStageName"),
            col("OpportunityStageId")
        )
    )
    
    # Map statecode to stage name using constants
    stage_name = (
        when(col("statecode").cast("long") == OPPORTUNITY_STATE_OPEN, lit(OPPORTUNITY_STAGE_NAME_OPEN))
        .when(col("statecode").cast("long") == OPPORTUNITY_STATE_WON, lit(OPPORTUNITY_STAGE_NAME_WON))
        .when(col("statecode").cast("long") == OPPORTUNITY_STATE_LOST, lit(OPPORTUNITY_STAGE_NAME_LOST))
        .otherwise(lit(None).cast(StringType()))
    )
    
    return (
        df.withColumn("OpportunityStageName", stage_name)
          .join(opportunity_stage, on="OpportunityStageName", how="left")
          .drop("OpportunityStageName")
    )


def transform_opportunity(df: DataFrame) -> DataFrame:
    """
    Transform Opportunity from Bronze to Silver.
    
    Business Logic:
    - OpportunityType: Resolved via opportunitysalesprocess (BPF workflow)
    - OpportunityStage: Mapped from statecode (0=Open, 1=Won, 2=Lost)
    - ConstituentId: Resolved via polymorphic customerid (Contact or Account)
    - CampaignId: Direct FK mapping
    """
    # Resolve OpportunityStage and OpportunityType
    df = resolve_opportunity_stage(df)
    df = resolve_opportunity_type(df)
    
    # Resolve customerid polymorphically (Contact or Account based on entitytype)
    df = data_sync.resolve_polymorphic_lookup(
        df,
        lookup_id_column="customerid",
        entity_type_column="customerid_entitytype",
        entity_type_mappings={
            "contact": "Contact",
            "account": "Account"
        }
    )
    
    # Resolve ConstituentId from Contact/Account
    df = data_sync.resolve_constituent_id(
        df,
        contact_fk_column="ContactId",
        account_fk_column="AccountId",
        output_column="ConstituentId"
    )
    
    return df.select(
        col("Id"),
        col("campaignid_SilverRecordId").alias("CampaignId"),
        col("actualclosedate").cast(TimestampType()).alias("CloseDate"),
        col("ConstituentId"),
        col("createdon").cast(TimestampType()).alias("CreatedDate"),
        col("estimatedvalue_base").cast(DecimalType(16, 8)).alias("ExpectedRevenue"),
        col("modifiedon").cast(TimestampType()).alias("ModifiedDate"),
        col("OpportunityStageId"),
        col("OpportunityTypeId"),
        substring(trim(col("name")).cast(StringType()), 1, MAX_NAME_LENGTH).alias("OpportunityName")
    )

data_sync.sync_table(
    source_table="opportunity",
    source_primary_key="Id",
    source_columns=[
        "Id", "campaignid", "customerid", "customerid_entitytype",
        "actualclosedate", "createdon", "modifiedon",
        "estimatedvalue_base", "name", "statecode"
    ],
    target_table="Opportunity",
    target_primary_key="OpportunityId",
    transform_func=transform_opportunity,
    fk_mappings={
        "campaignid": "Campaign"
        # customerid resolved dynamically in transform_func based on customerid_entitytype
    }
)

### Transform: Phonecall

In [ ]:
from pyspark.sql import DataFrame
from pyspark.sql.functions import col
from pyspark.sql.types import TimestampType

def transform_phonecall(df: DataFrame) -> DataFrame:
    """
    Transform phonecall to Silver via activityparty explosion (1 phonecall → N recipients).
    Resolves CampaignId and ConstituentId using Activity Helper Functions.
    """
    activityparty = data_sync.reader.read_activity_parties(
        activity_id_column="activityid",
        participation_types=PARTICIPATION_TYPES_SENDER_RECIPIENT
    )
    
    with_parties = (
        df.select(
            col("Id").alias("activityid"),
            col("createdon").cast(TimestampType()).alias("CreatedDate"),
            col("modifiedon").cast(TimestampType()).alias("ModifiedDate"),
            col("actualstart").cast(TimestampType()).alias("CallDate"),
            col("description").alias("Description"),
            col("regardingobjectid"),
            col("regardingobjectid_entitytype")
        )
        .join(activityparty, on="activityid", how="inner")
    )
    
    with_campaign = resolve_campaign_for_activity(with_parties)
    with_constituent = resolve_activity_party_constituent(
        with_campaign,
        party_id_col="partyid",
        party_type_col="partyid_entitytype"
    )
    
    channel_phonecall = lookup_channel_by_name(CHANNEL_PHONE_CALL)
    
    return (
        with_constituent.crossJoin(channel_phonecall)
        .select(
            col("ActivityPartyId").alias("SourceSystemId"),
            "CampaignId",
            "ConstituentId",
            "ChannelId",
            "CreatedDate",
            "ModifiedDate",
            "CallDate",
            "Description"
        )
        .filter(col("ConstituentId").isNotNull())
    )

# WHY composite key: Phonecall can have same ActivityPartyId from different sources,
# and same constituent can receive multiple phonecalls
phonecall_merge_sql = """
MERGE INTO {TARGET_TABLE} AS target
USING {SOURCE_VIEW} AS source
  ON  target.SourceId = source.SourceId
  AND target.SourceSystemId = source.SourceSystemId
  AND target.ConstituentId = source.ConstituentId

WHEN MATCHED THEN UPDATE SET
    target.ModifiedDate = source.ModifiedDate,
    target.CallDate = source.CallDate,
    target.Description = source.Description,
    target.CampaignId = source.CampaignId,
    target.ChannelId = source.ChannelId

WHEN NOT MATCHED THEN INSERT (
    PhonecallId, CampaignId, ChannelId, ConstituentId, CreatedDate,
    ModifiedDate, CallDate, Description, SourceId, SourceSystemId
) VALUES (
    source.PhonecallId, source.CampaignId, source.ChannelId, source.ConstituentId,
    source.CreatedDate, source.ModifiedDate, source.CallDate, source.Description,
    source.SourceId, source.SourceSystemId
)
"""

data_sync.sync_activity_table(
    source_table="phonecall",
    source_primary_key="Id",
    source_columns=[
        "Id", "createdon", "modifiedon", "actualstart", "description",
        "regardingobjectid", "regardingobjectid_entitytype"
    ],
    target_table="Phonecall",
    target_primary_key="PhonecallId",
    activity_party_join_func=transform_phonecall,
    merge_sql=phonecall_merge_sql
)

### Transform: Transaction

In [ ]:
from pyspark.sql import DataFrame
from pyspark.sql.functions import col
from pyspark.sql.types import TimestampType

def transform_transaction(df: DataFrame) -> DataFrame:
    """Transform transaction to Silver. Resolves ConstituentId via Contact/Account FKs."""
    return (
        data_sync.resolve_constituent_id(
            df,
            contact_fk_column="msnfp_receiptoncontactid_SilverRecordId",
            account_fk_column="msnfp_transaction_receiptonaccountid_SilverRecordId",
            output_column="ConstituentId"
        )
        .select(
            col("Id"),
            col("msnfp_amount").alias("Amount"),
            col("msnfp_name").alias("Name"),
            col("msnfp_bookdate").cast(TimestampType()).alias("TransactionDate"),
            col("createdon").cast(TimestampType()).alias("CreatedDate"),
            col("modifiedon").cast(TimestampType()).alias("ModifiedDate"),
            col("ConstituentId")
        )
    )

data_sync.sync_table(
    source_table="msnfp_transaction",
    source_primary_key="Id",
    source_columns=[
        "Id",
        "msnfp_transactionid",
        "msnfp_amount",
        "msnfp_name",
        "msnfp_bookdate",
        "msnfp_receiptoncontactid",
        "msnfp_transaction_receiptonaccountid",
        "createdon",
        "modifiedon"
    ],
    target_table="Transaction",
    target_primary_key="TransactionId",
    transform_func=transform_transaction,
    fk_mappings={
        "msnfp_receiptoncontactid": "Contact",
        "msnfp_transaction_receiptonaccountid": "Account"
    }
)

### Transform: Program

In [ ]:
from pyspark.sql import DataFrame
from pyspark.sql.functions import col
from pyspark.sql.types import TimestampType

def transform_program(df: DataFrame) -> DataFrame:
    """Transform msnfp_designation to Silver Program."""
    return df.select(
        col("Id"),
        col("createdon").cast(TimestampType()).alias("CreatedDate"),
        col("modifiedon").cast(TimestampType()).alias("ModifiedDate"),
        col("msnfp_name").alias("Name")
    )

data_sync.sync_table(
    source_table="msnfp_designation",
    source_primary_key="Id",
    source_columns=[
        "Id", "createdon", "modifiedon", "msnfp_name"
    ],
    target_table="Program",
    target_primary_key="ProgramId",
    transform_func=transform_program
)

### Transform: ConstituentProgram

In [ ]:

from pyspark.sql import DataFrame
from pyspark.sql.functions import col, expr, lit, lower, coalesce
from pyspark.sql.types import TimestampType, StringType

def _load_transaction_lookup_data() -> DataFrame:
    """
    Load transaction data needed to resolve constituent references.
    
    WHY: Designated credits don't directly reference Contact/Account.
         They link to Transactions, which hold the constituent reference
         (msnfp_receiptoncontactid or msnfp_transaction_receiptonaccountid).
    """
    return data_sync.reader.read_bronze_table(
        "msnfp_transaction",
        columns={
            "Id": "TransactionId",
            "msnfp_receiptoncontactid": "TransactionContactSourceId",
            "msnfp_transaction_receiptonaccountid": "TransactionAccountSourceId"
        }
    )

def _prepare_designated_credit_base(df: DataFrame) -> DataFrame:
    """
    Extract and normalize designated credit columns.
    
    WHY: Dynamics 365 uses polymorphic lookups (ID + entitytype).
         We need to validate that msnfp_transactionid points to msnfp_transaction
         and msnfp_designatiedcredit_designationid points to msnfp_designation.
    """
    return df.select(
        col("Id").alias("DesignatedCreditSourceId"),
        col("msnfp_transactionid").alias("DesignatedCreditTransactionId"),
        lower(coalesce(col("msnfp_transactionid_entitytype"), lit(""))).alias("DesignatedCreditTransactionEntity"),
        col("msnfp_designatiedcredit_designationid").alias("DesignatedCreditDesignationId"),
        lower(coalesce(col("msnfp_designatiedcredit_designationid_entitytype"), lit(""))).alias("DesignatedCreditDesignationEntity"),
        col("createdon").cast(TimestampType()).alias("CreatedDate"),
        col("modifiedon").cast(TimestampType()).alias("ModifiedDate")
    )

def _filter_valid_designated_credits(df: DataFrame) -> DataFrame:
    """
    Keep only designated credits with valid transaction and designation references.
    
    WHY: Polymorphic lookups can point to any entity type.
         We only want records where:
         - msnfp_transactionid points to msnfp_transaction (not other entities)
         - msnfp_designatiedcredit_designationid points to msnfp_designation (not other entities)
    """
    return (
        df.filter((col("DesignatedCreditTransactionId").isNotNull()) & 
                  (col("DesignatedCreditTransactionEntity") == "msnfp_transaction"))
          .filter((col("DesignatedCreditDesignationId").isNotNull()) & 
                  (col("DesignatedCreditDesignationEntity") == "msnfp_designation"))
    )

def _join_transaction_for_constituent_reference(df: DataFrame, tx: DataFrame) -> DataFrame:
    """
    Join transaction data to get Contact/Account references needed for constituent resolution.
    
    WHY: Designated credits track "which program received credit from which transaction".
         To know which constituent gave the donation, we need to follow:
         DesignatedCredit → Transaction → Contact/Account → Constituent
    """
    return df.join(tx, col("DesignatedCreditTransactionId") == col("TransactionId"), "left")

def _prepare_for_constituent_resolution(df: DataFrame) -> DataFrame:
    """
    Reshape data to match framework's resolve_constituent_id expectations.
    
    WHY: Framework expects specific column names for FK resolution:
         - msnfp_receiptoncontactid (Contact FK from transaction)
         - msnfp_transaction_receiptonaccountid (Account FK from transaction)
         - msnfp_designatiedcredit_designationid (Program FK, needs FK mapping)
    """
    return df.select(
        col("DesignatedCreditSourceId"),
        col("DesignatedCreditDesignationId").alias("msnfp_designatiedcredit_designationid"),
        col("TransactionContactSourceId").alias("msnfp_receiptoncontactid"),
        col("TransactionAccountSourceId").alias("msnfp_transaction_receiptonaccountid"),
        col("CreatedDate"),
        col("ModifiedDate")
    )

def _build_constituent_program_output(df: DataFrame) -> DataFrame:
    """
    Build final ConstituentProgram schema with synthetic PK and deduplication.
    
    WHY: ConstituentProgram is a link table (Constituent ↔ Program).
         - ConstituentProgramId: New synthetic UUID for Silver table
         - SourceSystemId: Original designated credit ID for tracking
         - Filter out records where constituent resolution failed (NULL ConstituentId)
         - Deduplicate by SourceSystemId (one ConstituentProgram per designated credit)
    """
    return (
        df.withColumn("ConstituentProgramId", expr("uuid()"))
          .select(
              "ConstituentProgramId",
              "ConstituentId",
              col("msnfp_designatiedcredit_designationid"),
              "CreatedDate",
              "ModifiedDate",
              col("DesignatedCreditSourceId").alias("SourceSystemId")
          )
          .filter(col("ConstituentId").isNotNull())
          .dropDuplicates(["SourceSystemId"])
    )

def transform_constituent_program(df: DataFrame) -> DataFrame:
    """
    Transform designated credit to ConstituentProgram.
    
    Business Logic:
    ConstituentProgram links constituents to programs they support.
    In Dynamics 365: Designation (Program) → DesignatedCredit → Transaction → Contact/Account → Constituent
    """
    # Step 1: Load transaction lookup data (holds constituent references)
    transactions = _load_transaction_lookup_data()
    
    # Step 2: Prepare base data from designated credits
    base = _prepare_designated_credit_base(df)
    
    # Step 3: Filter to valid transaction and designation references
    valid = _filter_valid_designated_credits(base)
    
    # Step 4: Join transaction data to get Contact/Account FKs
    with_transactions = _join_transaction_for_constituent_reference(valid, transactions)
    
    # Step 5: Prepare data for constituent resolution
    prepared = _prepare_for_constituent_resolution(with_transactions)
    
    # Step 6: Resolve ConstituentId via Contact/Account using framework
    with_constituent = data_sync.resolve_constituent_id(
        prepared,
        contact_fk_column="msnfp_receiptoncontactid",
        account_fk_column="msnfp_transaction_receiptonaccountid",
        output_column="ConstituentId"
    )
    
    # Step 7: Build final output schema
    return _build_constituent_program_output(with_constituent)

data_sync.sync_table(
    source_table="msnfp_designatedcredit",
    source_primary_key="Id",
    source_columns=[
        "Id",
        "msnfp_transactionid", "msnfp_transactionid_entitytype",
        "msnfp_designatiedcredit_designationid", "msnfp_designatiedcredit_designationid_entitytype",
        "createdon", "modifiedon"
    ],
    target_table="ConstituentProgram",
    target_primary_key="ConstituentProgramId",
    transform_func=transform_constituent_program,
    fk_mappings={
        "msnfp_designatiedcredit_designationid": "Program"
    }
)